In [1]:
#!pip install mysql-connector-python

In [30]:
import logging

# Set up basic logging
logging.basicConfig(
    filename='etl_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [31]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime

In [32]:
csv_file_path = 'employees1.csv'
df = pd.read_csv(csv_file_path)

print("Raw data loaded:")
print(df.head())
print(df.columns.tolist())

logging.info("CSV loaded successfully.")

Raw data loaded:
   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          198     Donald   OConnell  DOCONNEL  650.507.9833  21-Jun-07   
1          199    Douglas      Grant    DGRANT  650.507.9844  13-Jan-08   
2          200   Jennifer     Whalen   JWHALEN  515.123.4444  17-Sep-03   
3          201    Michael  Hartstein  MHARTSTE  515.123.5555  17-Feb-04   
4          202        Pat        Fay      PFAY  603.123.6666  17-Aug-05   

     JOB_ID  SALARY  
0  SH_CLERK    2600  
1  SH_CLERK    2600  
2   AD_ASST    4400  
3    MK_MAN   13000  
4    MK_REP    6000  
['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE', 'JOB_ID', 'SALARY']


In [33]:
df.fillna({
    'EMAIL': 'not_provided@example.com',
    'PHONE_NUMBER': '0000000000',
    'HIRE_DATE': '01-Jan-00',
    'SALARY': 0
}, inplace=True)

In [34]:
# Standardize column names
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
print(df.columns.tolist())

['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary']


In [35]:
# Convert 'hire_date' to proper date format
df['hire_date'] = pd.to_datetime(df['hire_date'], format='%d-%b-%y', errors='coerce')

# Replace invalid dates with default
df['hire_date'] = df['hire_date'].fillna(pd.to_datetime('2000-01-01'))

# Convert salary to integer, replace non-numeric
df['salary'] = pd.to_numeric(df['salary'], errors='coerce').fillna(0).astype(int)

logging.info("Data cleaning completed.")

In [36]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",         # or 127.0.0.1
    user="root",              # as shown under your connection
    password="Ashwin@2005",  # 🔑 use the one you typed to open Workbench
    database="employee"       # the database name (already created)
)

if mydb.is_connected():
    print("✅ Connected to MySQL Workbench!")


✅ Connected to MySQL Workbench!


In [37]:
mydb

In [38]:
cursor=mydb.cursor()

In [39]:
#sql="CREATE DATABASE employee"

In [40]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS salary_2 (
        empid INT,
        firstname VARCHAR(50),
        lastname VARCHAR(50),
        email VARCHAR(100),
        phone VARCHAR(20),
        hire_date DATE,
        job_id VARCHAR(20),
        salary INT
    )
""")

print("✅ Table created (if it didn't exist).")


✅ Table created (if it didn't exist).


In [41]:
#print("🧾 Actual columns in DataFrame:")
#print(df.columns.tolist())
#df.columns = [col.strip().lower() for col in df.columns]

df['hire_date'] = pd.to_datetime(df['hire_date'], errors='coerce')

for index, row in df.iterrows():
    sql = """
        INSERT INTO salary_2 (empid, firstname, lastname, email, phone, hire_date, job_id, salary)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            firstname=VALUES(firstname),
            lastname=VALUES(lastname),
            email=VALUES(email),
            phone=VALUES(phone),
            hire_date=VALUES(hire_date),
            job_id=VALUES(job_id),
            salary=VALUES(salary)
    """
    values = (
        int(row['employee_id']),
        row['first_name'],
        row['last_name'],
        row['email'],
        row['phone_number'],
        row['hire_date'].date(),  # ✅ Now this will work
        row['job_id'],
        int(row['salary'])
    )
    cursor.execute(sql, values)

mydb.commit()
cursor.close()
mydb.close()

logging.error("Something went wrong", exc_info=True)
print("✅ ETL process completed successfully.")

✅ ETL process completed successfully.


In [45]:
import os
print(os.listdir())
print(os.getcwd())

get_ipython().system('jupyter nbconvert --to script "ETL-Data Analysis.ipynb"')


['.anaconda', '.bash_history', '.conda', '.condarc', '.config', '.continuum', '.git', '.gitconfig', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.ms-ad', '.spyder-py3', '.vscode', 'anaconda3', 'Analysis of electricity usage.ipynb', 'AppData', 'Application Data', 'Assessment-1.ipynb', 'Contacts', 'Cookies', 'CrossDevice', 'Desktop', 'Documents', 'Downloads', 'Dropbox', 'Duplicate ID Detector using NumPy.ipynb', 'emergency-app', 'Employee hours analysis.ipynb', 'employees1.csv', 'employee_working_hours.csv', 'ETL-Data Analysis.ipynb', 'etl_log.txt', 'Favorites', 'Links', 'Local Settings', 'Music', 'My Documents', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{537fcc4d-2a3e-11f0-affb-c62ee5c7937e}.TM.blf', 'NTUSER.DAT{537fcc4d-2a3e-11f0-affb-c62ee5c7937e}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{537fcc4d-2a3e-11f0-affb-c62ee5c7937e}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'Numpy in array.ipynb', '

[NbConvertApp] Converting notebook ETL-Data Analysis.ipynb to script
[NbConvertApp] Writing 3406 bytes to ETL-Data Analysis.py
